In [1]:
solver_grammar = """
    start: _NL? problem
    problem: ([expr | decl] _NL)+
    decl: symbol "=" expr
    expr: binoperation
        | method
        | symbol
        | literal
    method: symbol "(" parameter ")"
    parameter: expr ("," expr)*
    binoperation: expr OPERATOR expr
    symbol: SYMBOL
    literal: LITERAL
    
    SYMBOL: /[a-zA-Z][a-zA-Z0-9]*/
    LITERAL: /\d*\.?\d+/
    OPERATOR: /[\/*\-+]/
    
    %import common.INT -> NUMBER
    %import common.NEWLINE -> _NL
    %import common.WS_INLINE
    
    %ignore WS_INLINE
"""

In [2]:
from lark import Lark
parser = Lark(solver_grammar)
sample = """
t = triangle(a, b, c)
d = triangle(z, x, y)
perimeter(d, 3)
perimeter(t, 2 * perimeter(d))
eq(dist(a, b), 2)
eq(dist(a, c), 2)
solve(dist(b, c))
"""
sample2 = """
t = triangle(a, b, c)
perimeter(t, 2 * 2)
eq(dist(a, b), 1)
eq(dist(a, c), 2)
solve(dist(b, c))
"""
sample3 = """
t = triangle(a,b,c)
perimeter(t, 5)
eq(dist(a,c),dist(b,c))
eq(dist(a,b),1)
solve(dist(a,c))
"""
sample4 = """
c = circle(a, r)
area(c, 314)
solve(r)
"""

sample5 = """
    t = triangle(a,b,c)
    c1 = mid(a,b)
    a1 = mid(b,c)
    b1 = mid(c,a)
    t1 = triangle(a1,b1,c1)
    perimeter(t1,13)
    eq(dist(a,b) / dist(b,c),3/4)
    eq(dist(b,c) / dist(c,a),4/6)
    solve(dist(a,b))
"""

sample6 = """
    t = triangle(a,b,c)
    a1 = mid(b,c)
    b1 = mid(c,a)
    m = intersect(a,a1,b,b1)
    solve(dist(a,m)/dist(m,a1))
"""

sample7 = """
    t = triangle(a,b,c)
    angle(a,c,b,90)
    dist(a,b) = 3
    dist(a,b) = 4
    h = altitude(t,c)
    solve(dist(a,h))
"""

print(parser.parse(sample4).pretty())

start
  problem
    decl
      symbol	c
      expr
        method
          symbol	circle
          parameter
            expr
              symbol	a
            expr
              symbol	r
    expr
      method
        symbol	area
        parameter
          expr
            symbol	c
          expr
            literal	314
    expr
      method
        symbol	solve
        parameter
          expr
            symbol	r



In [3]:
from problem import *
from lark import Tree, Token

p = Problem() 

methods = {
    'triangle': p.triangle,
    'circle': p.circle,
    'perimeter': p.perimeter,
    'area': p.area,
    'eq': p.eq,
    'dist': p.dist,
    'solve': p.solve,
    'mid': p.midpoint,
    'intersect': p.intersect,
    'altitude': p.altitude,
    '+': p.add,
    '-': p.sub,
    '*': p.mul,
    '/': p.div
}

def process_expression(root, problem):
    if root.data == 'literal':
        return(float(root.children[0]))
    elif root.data == 'symbol':
        return(str(root.children[0]))
    elif root.data == 'binoperation':
        res = visit_branch(root, problem) 
        return (methods.get(res[1], lambda: 'Invalid')(res[0], res[2]))
    elif root.data == 'method':
        method = visit_branch(root, problem)
        return(methods.get(method[0], lambda: 'Invalid')(*method[1]))

    #return visit_branch(root)

def visit_branch(root, problem):
    if isinstance(root, Tree):
        params = [visit_branch(child, problem) for child in root.children]
       # print(params)
        if root.data == 'expr':
            return process_expression(root.children[0], problem)
        elif root.data == 'decl':
            problem.bind(*params)
        elif root.data == 'literal' or root.data == 'symbol':
            return params[0]
        return params
    if isinstance(root, Token):
        return str(root)
               
visit_branch(parser.parse(sample7), p)

UnexpectedCharacters: No terminal defined for '=' at line 4 col 15

    dist(a,b) = 3
              ^

Expecting: {Terminal('_NL'), Terminal('OPERATOR')}
